# Coursework 2

We first need to setup the python environment, import libraries, declare common functions, etc.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# Constants
TRADING_YEAR = 252  # Number of trading days in a year

# Basic helper functions
def percent(x): return x / 100
def seed(n): return np.random.RandomState(n)

The `monte_carlo(μ, σ, price, days, iterations, rng)` function below simulates a share price defined by the equation $\frac{P_t}{P_{t-1}}=Z$ where $P_t$ is the price at time $t$ and $Z$ is a random draw from a normal distribution with a mean passed as $\mu$ and a standard deviation passed as $\sigma$.
_price_ is the initial price of the share on day 1, _days_ is the number of days to run the simulation for, _iterations_ is the number of times to repeat the simulation, and _rng_ is a [`numpy.random.Generator`](https://numpy.org/doc/stable/reference/random/generator) to supply the random numbers (e.g. if you need a fixed seed).

In [ ]:
def monte_carlo(μ=0, σ=1, price=100, days=252, iterations=1, rng=None):
    μ = percent(np.atleast_1d(μ))
    σ = percent(np.atleast_1d(σ))
    variations = max(μ.size, σ.size)
    if rng is None:
        rng = seed(None)
    shape = (days, variations, iterations)
    Z = np.empty(shape)
    for i in range(variations):
        Z[:,i,] = rng.normal(loc=1+μ[i%μ.size], scale=σ[i%σ.size], size=shape[::2])
    P = np.empty(shape)
    P[0,...] = price
    for i in range(1, days):
        P[i,...] = P[i-1,...] * Z[i,...]
    return np.squeeze(P)

These small functions allow the calculation of profit from a put or call option on a whole array at once. _Pt_ can be a scalar or an array, whereas _X_ should be a scalar.

In [ ]:
def put_option(X, Pt):
    return np.maximum(X - Pt, 0)

def call_option(X, Pt):
    return np.maximum(Pt - X, 0)

The `bisect(callback, target, floor, ceiling)` function below will be used for questions 2c & 3c.

_callback_ should be a callable which takes takes and returns a number.

_target_ is the output we are attempting to approximate.

_floor_ and _ceiling_ are numbers which should return a number lower and a number higher than _target_ when passed to _callback_.

In [ ]:
def bisect(callback, target, floor=1, ceiling=0, precision=0.5):
    # Try a value between floor and ceiling
    μ = (floor+ceiling)/2
    x = callback(μ)
    # Check if the result is close to the target
    if np.isclose(x, target, atol=precision):  # Tweaked target & precision values would yield much closer approximations
        return μ
    # ...otherwise recurse. Try a number between μ and ceiling if we're too low, between floor and μ if we're too high
    if x < target:
        return bisect(callback, target, μ, ceiling, precision)
    return bisect(callback, target, floor, μ, precision)

## Question 1

Plots the share price of the 3 companies over 756 days:

In [ ]:
# Simulate the share price of the 3 companies
P = monte_carlo(μ=(0.1, 0.2, 0.3), σ=(1, 4, 6), days=756, rng=seed(345))

# Create a figure and plot to output to
figure, plot = plt.subplots()

# Resize the figure from the small default size
figure.set_size_inches(12, 9)

# Set x & y axis limits
plot.set_xlim(0,756)
plot.set_ylim(0,400)

# Set x axis ticks to nicer values; major each trading year, minor each quarter
plot.xaxis.set_major_locator(plt.MultipleLocator(252))
plot.xaxis.set_minor_locator(plt.MultipleLocator(63))

# Turn on grid lines
plot.grid(which='both')

# Draw the share price over time of the companies to the plot
plot.plot(P[...,0], label='Company 1: μ=0.1% σ=1%')
plot.plot(P[...,1], label='Company 2: μ=0.2% σ=4%')
plot.plot(P[...,2], label='Company 3: μ=0.3% σ=6%')

# Add a title, axis labels, and legend to the plot
plot.set_title('756-day simulation of 3 companies')
plot.set_xlabel('Days')
plot.set_ylabel('Projected Price')
plot.legend()

pass # Prevent ugly matplotlib output

## Question 2

The three parts of this question rely on calculating the mean price of a put option based the same simulation with slightly modified parameters, so we define the `question2_mean()` function to facilitate this:

In [ ]:
def question2_mean(μ=0.05, X=175):
    P = monte_carlo(μ, σ=0.1, days=756, iterations=10_000, rng=seed(678))[-1]  # Slice off just the final price value
    return np.mean(put_option(X, P))  # Just return the mean as this is all that is needed for this question

### Question 2a

Calculates the mean payoff of a put option with a strike price of 150 over 10,000 756-day simulations with a starting price of 100 where $\mu=0.05\%$ and $\sigma=0.1\%$.  

In [ ]:
p1 = question2_mean(X=150)
print(f'Mean profit: {p1:.2f}')

### Question 2b

The same as above, but with a strike price of 175.

In [ ]:
p2 = question2_mean(X=175)
print(f'Mean profit: {p2:.2f}')
print(f'Change: {p2-p1:+.2f}')

### Question 2c
To approximate a value for $\mu$ which will yield a mean profit of around $4.49$ given an $X$ of $175$ we can use bisection;
We will need an intial floor value for $\mu$ which we know yields a mean profit which is lower than $4.49$, and an initial ceiling value which yields a mean profit higher than $4.49$.
A ceiling value in this instance is easy as we already know that $\mu=0.05$ yields a considerably higher mean profit.
A floor value can be worked out by considering that mean profit would be $0$ at _approximately_ the point where $P_{756}$ averages to $175$.
This can be calculated with the equation $\mu=100(\sqrt[756]{\frac{175}{100}}-1)$ which yields approximately $0.074$.
This should probably be checked to ensure it is actually lower than the $4.49$ target:

In [ ]:
print(f'{question2_mean(μ=0.074):.2f} is lower than 4.49')

Given our floor & ceiling values of $0.074$ & $0.05$ we can recursively check if an arbitrary value _between_ these two numbers yields our target of $4.49$, adjusting our floor or ceiling value (to match the number we just tried) after each attempt depending if the result is lower or higher than our target.

A function for bisection, `bisect()`, is defined earlier in the notebook, and will calculate a value for $\mu$ which will _approximately_ yield our $4.49$ target when passed `question2_mean()`, the target of $4.49$ (to the nearest integer), and our floor & ceiling values:

In [ ]:
# Run the bisection and print the result to 3 decimal places:
μ = bisect(callback=question2_mean, target=round(4.49), floor=0.074, ceiling=0.05)
print(f'μ = {μ:.3f} should approximate the payoff from Question 2a.\n')

# Print what this value of μ actually yields:
x = question2_mean(μ)
print(f'Mean payoff when μ = {μ:.3f} is {x:.2f}, which is {x:.0f} when rounded to the nearest integer.')

## Question 3


The three parts of this question rely on calculating the mean price of an option based the same simulation with slightly modified parameters, so we define the `question3_mean()` function to facilitate this:

In [ ]:
def question3_mean(μ=0.05, X=140):
    # Get the mean value of the share over 504 days
    P = np.mean(monte_carlo(μ, σ=0.1, days=504, iterations=10_000, rng=seed(678)), 0)
    
    # put_option() operates on all 10,000 values at once
    P = put_option(X, P)
    
    # Only the mean of all 10,000 simulations is actually needed
    return np.mean(P)

### Question 3a

Calculate mean payoff where $X=150$:

In [ ]:
p1 = question3_mean(X=150)
print(f'Mean profit: {p1:.2f}')

### Question 3b

Calculate change in mean payoff if $X=140$ instead:

In [ ]:
p2 = question3_mean(X=140)
print(f'Mean profit: {p2:.2f}')
print(f'Change: {p2-p1:+.2f}')

### Question 3c

Bisection can again be used to approximate a value for $\mu$, in this case one that will yield a mean payoff which is around $36.30$ where $X=140$.
This time we already have a value for the floor as we know $\mu=0.05$ yields a result lower than our target, so we need to find a value for the ceiling instead.
As $P_0=100$ and $X=140$ a lack of change should result in a mean payoff of $40$ (which is higher than our target), thus $\mu=0$ should work for our ceiling value.
This can be checked before we run the bisection:

In [ ]:
print(f'{question3_mean(μ=0):.2f} is higher than 36.30')

We can then run the bisection to calculate a value for $\mu$ that will yield approximately $36.30$:

In [ ]:
# Run the bisection and print the result to 3 decimal places:
μ = bisect(callback=question3_mean, target=round(36.30), floor=0.05, ceiling=0)
print(f'μ = {μ:.3f} should approximate the payoff from Question 3a.\n')

# Print what this value of μ actually yields:
x = question3_mean(μ)
print(f'Mean payoff when μ = {μ:.3f} is {x:.2f}, which is {x:.0f} when rounded to the nearest integer.')

## Question 4

I'm running two simulations 100,000 times each using starting price $P_0=100$, daily mean $\mu=0.01\%$, and standard deviation $\sigma=0.1\%$.
The first will run for 5 trading years, or 1260 days, and the second for 10 trading years, or 2520 days.

In [ ]:
# Set common parameters
μ=0.01
σ=0.1
iterations=10_000
rng_seed=678

# Get final share values after 5 and 10 years
P5 = monte_carlo(μ, σ, days=5*TRADING_YEAR, iterations=iterations, rng=seed(rng_seed))[-1]
P10 = monte_carlo(μ, σ, days=10*TRADING_YEAR, iterations=iterations, rng=seed(rng_seed))[-1]

# Print out closing prices and call option payouts
print(f'Mean closing price at T = 5 years: {np.mean(P5):.2f}')
print(f'Mean closing price at T = 10 years: {np.mean(P10):.2f}\n')

print(f'Mean payout of call option at T = 5 years: {np.mean(call_option(100, P5)):.2f}')
print(f'Mean payout of call option at T = 10 years: {np.mean(call_option(100, P10)):.2f}')

The payout of a call option increases as the final value of the share does, so the mean payout of the option is positively correlated with the time to maturity $T$: because the variance of $\sigma$ effectively cancels out over a large number of simulations the mean price of the share at any given time $P_t$ tends towards simply $P_t = P_0(1+\frac{\mu}{100})^{t-1}$ which, given a positive value of $\mu$, will have a positive trend.